# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe.

HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, I used the Denver meeting minutes, or you can record something of your own!

In [ ]:
# pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

Note: you may need to restart the kernel to use updated packages.


In [6]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from dotenv import load_dotenv

In [7]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [8]:
# Loading in audio

audio_filename = "denver_extract.mp3"

if os.path.exists(audio_filename):
    print("✅ Audio file found")
else:
    print("❌ Audio file not found. Check the path.")


✅ Audio file found


In [12]:
# Load the .env file
load_dotenv()

# Read and login with tokens
hf_token = os.getenv("HF_TOKEN")
if hf_token is None:
    raise ValueError("HF_TOKEN not found in .env file")
login(hf_token)

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY not found in .env file")
OpenAI.api_key = openai_api_key

openai = OpenAI(api_key=openai_api_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [13]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism and you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United Sta

In [14]:
system_message = (
    "You are an expert assistant that generates professional meeting minutes from transcripts. "
    "Your output should be in clean, structured markdown format and must include the following sections: "
    "1) Meeting Summary with attendees, location, and date, "
    "2) Key Discussion Points, "
    "3) Notable Takeaways, and "
    "4) Action Items with assigned owners."
)

user_prompt = (
    f"Here is an excerpt from a Denver city council meeting transcript. "
    f"Please analyze the content and produce well-organized meeting minutes in markdown format. "
    f"Be sure to include:\n"
    f"- A summary with attendees, location, and meeting date\n"
    f"- A list of key discussion points\n"
    f"- Important takeaways\n"
    f"- Action items clearly assigned to individuals or groups\n\n"
    f"---\n\n{transcription}"
)

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [15]:
quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_compute_dtype=torch.bfloat16,
    bnb_8bit_quant_type="nf4"
)

In [16]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert assistant that generates professional meeting minutes from transcripts. Your output should be in clean, structured markdown format and must include the following sections: 1) Meeting Summary with attendees, location, and date, 2) Key Discussion Points, 3) Notable Takeaways, and 4) Action Items with assigned owners.<|eot_id|><|start_header_id|>user<|end_header_id|>

Here is an excerpt from a Denver city council meeting transcript. Please analyze the content and produce well-organized meeting minutes in markdown format. Be sure to include:
- A summary with attendees, location, and meeting date
- A list of key discussion points
- Important takeaways
- Action items clearly assigned to individuals or groups

---

and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politi

In [17]:
response = tokenizer.decode(outputs[0])

In [18]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert assistant that generates professional meeting minutes from transcripts. Your output should be in clean, structured markdown format and must include the following sections: 1) Meeting Summary with attendees, location, and date, 2) Key Discussion Points, 3) Notable Takeaways, and 4) Action Items with assigned owners.<|eot_id|><|start_header_id|>user<|end_header_id|>

Here is an excerpt from a Denver city council meeting transcript. Please analyze the content and produce well-organized meeting minutes in markdown format. Be sure to include:
- A summary with attendees, location, and meeting date
- A list of key discussion points
- Important takeaways
- Action items clearly assigned to individuals or groups

---

and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism and you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. All right. Thank you, Councilman Lopez. Madam Secretary, roll call. Clerk. Here. Espinosa. Here. Flynn. Here. Gilmour. Here. Here. Cashman. Here. Kenneche. Here. Lopez. Here. New. Here. Ortega. Here. Sussman. Here. Mr. President. Here. 11 present. 11 members present. We do have a quorum. Approval of the minutes. Seeing none. Minutes of October 2nd stand approved. Council announcements. Are there any announcements by members of Council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock p.m. It will move along Broadway from 3rd to Alameda. It's going to be a fun, family-friendly event. Everyone's invited to come down, wear a costume. There will be candy for the kids and there are tiki zombies and 29 hearses and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at 6 o'clock for the Broadway Halloween parade. Thank you, Mr. President. All right. Thank you, Councilman Clark. I will be there. All right. Presentations. Madam Secretary, do we have any presentations? None, Mr. President. Communications. Do we have any communications? None, Mr. President. We do have one proclamation this evening. Proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President, with pride. Proclamation number 17, well, let me just say this differently. Proclamation number 1127, series of 2017, an observance of the second annual Indigenous Peoples Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the Indigenous Peoples have lived and flourished on the lands known as the Americas since time immemorial and that Denver and the surrounding communities are built upon the ancestral homelands of numerous Indigenous tribes, which include the Southern Ute, the Ute Mountain, Ute tribes of Colorado. And whereas the tribal homelands and seasonal encampments of the Arapaho and Cheyenne people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes and the City and County of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas on October 3, 2016, the City and County of Denver unanimously passed Council Bill 801, series of 2016, officially designating the second Monday of October of each year as Indigenous Peoples Day in Denver, Colorado. And whereas the Council of the City and County of Denver continues to recognize and value the vast contributions made to the community through Indigenous Peoples' knowledge, science, philosophy, arts, and culture. And through these contributions, the City of Denver has developed and thrived. Whereas the Indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of Indigenous people, including Confluence Week, drawing record attendance to a National Indigenous Youth Leadership Conference, leading conversations on inclusion with their peers, and supporting increased Indigenous youth participation in science and engineering. Now, therefore, be it proclaimed by the Council of the City and County of Denver, Section 1, that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of Indigenous people to our history, our past, our present, and future, and continues to promote the education of the Denver community about these historical and contemporary contributions of Indigenous people. Section 2, that the City and County of Denver, Colorado, does hereby observe October 9, 2017 as Indigenous Peoples Day. Section 3, that the Clerk of the City and County of Denver shall attest and affix the seal of the City and County of Denver to this proclamation, and that a copy be transmitted to the Denver American Indian Commission, the City and County of Denver School District No. 1, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez. Your motion to adopt. Mr. President, I move that Proclamation No. 1127, Series of 2017, be adopted. All right. It has been moved and seconded. It comes from the members of Council. Councilman Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver, officially for the second time. It's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our Council Chambers. It was a very beautiful piece of artwork that you presented to us earlier, and it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous Peoples Day. And we sat down and wrote it, and as a community, we couldn't think of anything else to begin except for the confluence of the two rivers. And those confluence of the two rivers created such a great city. And we live in such an amazing city, and we're all proud of it, and sometimes we, and a lot of people from all over the country or all over the world are proud of it, and sometimes a little too proud of it is to tell them to go back home. But I'm kidding when I say that. But the really nice thing about this is that we are celebrating Indigenous Peoples Day out of pride for who we are, who we are as a city, and the contributions of Indigenous Peoples to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt or disrespect. I think of a quote that Cesar Chavez made very popular, and it stuck with me for a very long time, and any time I have the opportunity to speak in front of children, and especially children in our community, that they often second-guess themselves on where they're coming from, who they are. And I always say that it's very important to be proud of where you're from, and the quote that I use from Cesar Chavez is, you know, pride in one's own culture does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that no matter who we are, where we come from in this society, your pride in your own culture does not require the contempt or disrespect of another. And man, what a year for that to just sit on our shoulders for a while, for us to think about, right? And so I wanted to just thank you all. I think the Commission, there's going to be a couple individuals that are going to come and speak. Thank you for your art, your lovely artwork, for us to see what's in your heart and what now has become, probably is going to be a very important symbol for the community. And also just for the work, the daily work, every single day. We still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, right, in poverty, without access to services, right, without access to sobriety or even housing or jobs. And what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do. And these kind of proclamations and this day is not a day off, it's a day on in Denver, right, in addressing those critical issues. So I know that my colleagues are very supportive. I'm going to ask you to support this proclamation, as I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again today as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Martega? Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was and had the benefit of being able to go down to the Four Corners for a peace treaty signing ceremony between the Utes and the Comanches that had been sort of at odds with each other for about 100 years, and just being able to participate in that powwow was pretty awesome. And for those of you who continue to participate in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the Native language from generation to generation is just so incredible, because in so many cultures, you know, people have come here and assimilated to the, you know, the norms here, and they lose their language and lose a lot of the culture. And in the Native community, that hasn't happened. That, you know, commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right. Thank you, Councilwoman Ortega. Councilwoman Knitsch. Thank you very much, and I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is, you know, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture, and so I think the art is so important. And when you talked about water, I was also thinking about land, and I guess I just wanted to say thank you. Many of the Native American peoples of Colorado have been at the forefront, or actually nationally, of defending some of the public lands that have been protected over the last few years that are under attack right now. And there are places that the communities have fought to protect but that everyone gets to enjoy, and so I just think that it's an example of where cultural preservation intersects with environmental protection, with recreation and all of the other ways that public lands are so important. And so I think I just wanted to say thank you for that, because I think we have some very sacred places in our country that are at risk right now. And so as we celebrate, I appreciate that there's still a piece of resistance in here, and I think that I just want to mention a feeling of solidarity with that resistance. So thank you, and happy Confluence Week. Thank you, Councilwoman Kinneach. And seeing no other comments, I'll just say a couple. And in a time of such divisive ugliness and just despicable behavior from our leadership, the reason I'm so supportive of Indigenous Peoples' Day is because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told. And so we celebrate inclusivity in the face of such evil times, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Meeting Summary**
================

*   **Attendees:** 
    *   Councilman Lopez
    *   Councilman Martega
    *   Councilwoman Ortega
    *   Councilwoman Kinneach
    *   Clerk
    *   Councilman Clark
    *   Councilman New
    *   Councilman Gilmour
    *   Councilman Flynn
    *   Councilman Espinosa
    *   Councilman Cashman
    *   Councilman Kenneche
    *   Councilman Sussman
    *   Councilman Mr. President
*   **Location:** Denver City Council Chambers
*   **Date:** Monday, October 9th

**Key Discussion Points**
=====================

*   The Denver City Council discussed the proclamation of Indigenous Peoples Day in the City and County of Denver.
*   Councilman Lopez presented the proclamation, highlighting the importance of recognizing and honoring the cultural and foundational contributions of Indigenous people to the city's history, past, present, and future.
*   The council members expressed their support for the proclamation and the celebration of Indigenous Peoples Day.
*   Councilwoman Kinnech mentioned the importance of cultural preservation and environmental protection, highlighting the efforts of Native American peoples in defending public lands.

**Notable Takeaways**
==================

*   The proclamation of Indigenous Peoples Day aims to promote education and awareness about the historical and contemporary contributions of Indigenous people in Denver.
*   The council members emphasized the importance of inclusivity, respect, and celebration of diversity in the face of divisive and ugly behavior.
*   The proclamation also recognizes the contributions of Indigenous people to the city's development and growth.

**Action Items**
==============

*   **Councilman Lopez:** 
    *   To ensure the proclamation is adopted and implemented.
    *   To work with the council members and the community to promote education and awareness about Indigenous Peoples Day.
*   **Councilwoman Kinnech:** 
    *   To continue supporting the efforts of Native American peoples in defending public lands and promoting cultural preservation.
    *   To work with the council members and the community to promote environmental protection and conservation.
*   **Councilman Martega:** 
    *   To continue supporting the proclamation and the celebration of Indigenous Peoples Day.
    *   To work with the council members and the community to promote cultural preservation and environmental protection.
*   **Councilman Clark:** 
    *   To invite the community to the first-ever Halloween parade on Broadway in Lucky District 7.
    *   To promote the event and encourage participation from the community.<|eot_id|>

### 🧠 Speech-to-Text Inference with Whisper ASR

In this section, we load and run OpenAI's Whisper model to transcribe speech from an audio file into text.

The pipeline includes the following steps:

1. **Model Setup**: We load the `openai/whisper-medium` model from Hugging Face, configure it for either GPU or CPU, and use efficient memory settings with `float16` where possible.
2. **Processor Initialization**: We load the processor, which includes the tokenizer and feature extractor required by the Whisper model.
3. **Pipeline Creation**: We assemble an `automatic-speech-recognition` pipeline using the Hugging Face `pipeline()` utility.
4. **Transcription Execution**: We pass a local audio file into the pipeline and extract the transcription result.
5. **Output**: The final transcription is printed in plain text for use in downstream tasks (e.g. summarization or meeting minute generation).

This setup allows us to efficiently convert audio to text with high accuracy using a production-ready ASR model.

> 📌 **Note:** Ensure your environment supports GPU (`cuda`) and that the audio file path is valid before running this section.


In [24]:
import torch
from transformers import (
    AutoModelForSpeechSeq2Seq,
    AutoProcessor,
    pipeline
)

# ===============================
# 🧠 Configuration and Setup
# ===============================

# ID of the OpenAI Whisper model from Hugging Face
AUDIO_MODEL_ID = "openai/whisper-medium"

# Automatically choose 'cuda' if available, otherwise fallback to 'cpu'
USE_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Use float16 for better performance on CUDA, else float32 for CPU
DTYPE = torch.float16 if USE_DEVICE == "cuda" else torch.float32

# ===============================
# 📦 Model & Processor Loading
# ===============================

print(f"🔄 Loading model: {AUDIO_MODEL_ID} on {USE_DEVICE.upper()}")

try:
    # Load the pre-trained Whisper model for speech-to-text
    speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(
        AUDIO_MODEL_ID,
        torch_dtype=DTYPE,
        low_cpu_mem_usage=True,     # Optimized loading for low-memory setups
        use_safetensors=True        # Use safer and faster tensor serialization
    ).to(USE_DEVICE)                # Move model to selected device (GPU or CPU)

    # Load the matching processor (tokenizer + feature extractor)
    processor = AutoProcessor.from_pretrained(AUDIO_MODEL_ID)

    # ===============================
    # 🔁 Pipeline Assembly
    # ===============================

    # Create an automatic speech recognition pipeline using the loaded components
    asr_pipeline = pipeline(
        task="automatic-speech-recognition",
        model=speech_model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=DTYPE,
        device=0 if USE_DEVICE == "cuda" else -1  # Use device index for pipeline
    )

    # ===============================
    # 🔊 Audio Transcription
    # ===============================

    # Provide the path to your local audio file
    print(f"🎧 Transcribing audio file: {audio_filename}")

    # Run the audio file through the pipeline to get the transcription
    result = asr_pipeline(audio_filename, return_timestamps=True)

    # Extract the transcribed text
    transcription = result["text"]

    # Print the final output
    print("\n📄 Transcription Output:\n")
    print(transcription)

# ===============================
# ⚠️ Error Handling
# ===============================
except Exception as e:
    print(f"❌ Error during ASR pipeline execution: {e}")


🔄 Loading model: openai/whisper-medium on CUDA


Device set to use cuda:0


🎧 Transcribing audio file: denver_extract.mp3


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.



📄 Transcription Output:

 and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind symbolism and you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. rise with the Pledge of Allegiance by Councilman Lopez all right thank you Councilman L

In [33]:
def display_transcription_with_timestamps(chunks):
    markdown_output = "### 🧾 Timestamped Transcript\n\n"
    for chunk in chunks:
        start, end = chunk['timestamp']
        text = chunk['text'].strip()
        markdown_output += f"**[{start:.2f}s - {end:.2f}s]**: {text}\n\n"
    display(Markdown(markdown_output))

# Show it
display_transcription_with_timestamps(result["chunks"])

### 🧾 Timestamped Transcript

**[0.00s - 4.72s]**: and kind of the confluence of this whole idea of the Confluence Week, the merging of two

**[4.72s - 5.72s]**: rivers.

**[5.72s - 13.72s]**: And as we've kind of seen recently in politics and in the world, there's a lot of situations

**[13.72s - 16.96s]**: where water is very important right now, and it's a very big issue.

**[16.96s - 21.96s]**: So that is the reason that the back of the logo is considered water.

**[21.96s - 0.00s]**: 

**[16.80s - 24.68s]**: So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind

**[24.68s - 0.00s]**: 

**[7.08s - 10.92s]**: symbolism and you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we

**[10.92s - 16.20s]**: can kind of bring more people together and kind of share this whole idea of Indigenous

**[16.20s - 17.20s]**: People's Day.

**[17.20s - 18.60s]**: So thank you.

**[18.60s - 21.64s]**: Thank you so much and thanks for your leadership.

**[21.64s - 23.52s]**: All right.

**[23.52s - 27.04s]**: Welcome to the Denver City Council meeting of Monday, October 9th.

**[27.04s - 0.00s]**: 

**[23.72s - 0.00s]**: rise with the Pledge of Allegiance by Councilman Lopez all right thank you

**[10.28s - 21.48s]**: Councilman Lopez, Madam Secretary roll call. Black. Clark. Here. Espinosa. Here. Flynn. Gilmour. Here. Here. Cashman. Here. Kenich. Here. Lopez. Here. New. Here. Ortega. Here. Sussman. Mr.

**[21.48s - 27.52s]**: President. Here. Eleven present. Eleven members present. We do have a quorum.

**[27.52s - 0.00s]**: 

**[1.40s - 5.88s]**: Approval of the minutes. Are there any corrections to the minutes of October 2nd?

**[5.88s - 8.64s]**: Seeing none, minutes of October 2nd stand approved.

**[8.64s - 9.68s]**: Council Announcements.

**[9.68s - 12.20s]**: Are there any announcements by members of council?

**[17.36s - 18.52s]**: Councilman Clark.

**[18.52s - 19.56s]**: Thank you, Mr. President.

**[19.56s - 21.24s]**: I just wanted to invite everyone down

**[21.24s - 25.28s]**: to the first ever Halloween Parade on Broadway

**[25.28s - 26.66s]**: in Lucky District 7.

**[26.66s - 29.20s]**: It will happen on Saturday, October 21st

**[29.20s - 0.00s]**: 

**[3.24s - 4.96s]**: At six o'clock p.m., it'll move along Broadway from Third to Alameda.

**[4.96s - 7.24s]**: It's gonna be a fun, family-friendly event.

**[7.24s - 10.94s]**: Everyone's invited to come down, wear a costume.

**[10.94s - 12.68s]**: There'll be candy for the kids,

**[12.68s - 17.68s]**: and there are tiki zombies and 29 hearses,

**[18.52s - 21.56s]**: and all kinds of fun and funky stuff

**[21.56s - 24.24s]**: on the fun and funky part of Broadway.

**[24.24s - 26.88s]**: So please join us October 21st at six o'clock

**[26.88s - 27.84s]**: for the Broadway Halloween.

**[27.84s - 29.32s]**: Brad, thank you, Mr. President.

**[29.32s - 0.00s]**: 

**[2.60s - 4.12s]**: All right, thank you, Councilman Clark. I will be there.

**[4.12s - 6.88s]**: All right, presentations.

**[6.88s - 8.56s]**: Madam Secretary, do we have any presentations?

**[8.56s - 9.68s]**: None, Mr. President.

**[9.68s - 11.76s]**: Communications, do we have any communications?

**[11.76s - 12.84s]**: None, Mr. President.

**[12.84s - 16.08s]**: We do have one proclamation this evening,

**[16.08s - 19.72s]**: Proclamation 1127, an observance of

**[19.72s - 22.76s]**: the annual Indigenous Peoples Day

**[22.76s - 24.12s]**: in the city and county of Denver.

**[24.12s - 27.04s]**: Councilman Lopez, will you please read it?

**[27.04s - 29.00s]**: Thank you, Mr. President, with pride.

**[0.00s - 4.12s]**: Proclamation number 17, well let me just say this differently.

**[4.12s - 8.72s]**: Proclamation number 1127, series of 2017, an observance

**[8.72s - 10.88s]**: of the second annual Indigenous Peoples Day

**[10.88s - 12.92s]**: in the City and County of Denver.

**[12.92s - 14.76s]**: Whereas the Council of the City and County of Denver

**[14.76s - 17.44s]**: recognizes that the Indigenous Peoples have lived

**[17.44s - 19.56s]**: and flourished on the lands known as the Americas

**[19.56s - 23.48s]**: since time immemorial, and that Denver

**[23.48s - 25.32s]**: and the surrounding communities are built upon

**[25.32s - 29.32s]**: the ancestral homelands of numerous Indigenous tribes,

**[29.32s - 0.00s]**: 

**[2.86s - 4.74s]**: which include the Southern Ute, the Ute Mountain, Ute Tribes of Colorado.

**[4.74s - 7.62s]**: And whereas the tribal homelands and seasonal encampments

**[7.62s - 10.66s]**: of the Arapahoe and Cheyenne people along the banks

**[10.66s - 13.64s]**: of the Cherry Creek and South Platte River confluence

**[13.64s - 15.34s]**: give bearing to the future settlements

**[15.34s - 18.14s]**: that would become the birthplace of the Mile High City.

**[19.62s - 23.44s]**: And whereas Colorado encompasses the ancestral homelands

**[23.44s - 26.00s]**: of 48 tribes in the city and county of Denver

**[26.00s - 28.00s]**: and surrounding communities are home

**[28.00s - 0.00s]**: 

**[3.94s - 7.86s]**: to the descendants of approximately 100 tribal nations. And whereas, on October 3rd, 2016,

**[7.86s - 11.38s]**: the City and County of Denver unanimously passed

**[11.38s - 14.34s]**: Council Bill 801, Series of 2016,

**[14.34s - 17.54s]**: officially designating the second Monday of October

**[17.54s - 20.78s]**: of each year as Indigenous Peoples' Day

**[20.78s - 21.96s]**: in Denver, Colorado.

**[23.06s - 25.86s]**: And whereas, the Council of the City and County of Denver

**[25.86s - 28.16s]**: continues to recognize and value

**[28.16s - 0.00s]**: 

**[5.00s - 5.24s]**: the vast contributions made to the community

**[7.40s - 12.06s]**: through indigenous people's knowledge, science, philosophy, arts, and culture,

**[12.06s - 13.60s]**: and through these contributions,

**[13.60s - 15.92s]**: the city of Denver has developed and thrived.

**[17.44s - 21.56s]**: Whereas, the indigenous community, especially you,

**[21.56s - 23.44s]**: have made great efforts this year

**[23.44s - 25.16s]**: to draw attention to the contributions

**[25.16s - 28.28s]**: of indigenous people, including Confluence Week,

**[28.28s - 0.00s]**: 

**[2.56s - 3.98s]**: Drawing record attendance to a national indigenous youth leadership conference,

**[3.98s - 7.56s]**: leading conversations on inclusion with their peers

**[7.56s - 10.92s]**: and supporting increased indigenous youth participation

**[10.92s - 13.24s]**: in science and engineering.

**[13.24s - 15.96s]**: Now therefore, be it proclaimed by this council

**[15.96s - 17.60s]**: of the city and county of Denver,

**[17.60s - 20.76s]**: section one, that the council of the city and county

**[20.76s - 22.80s]**: of Denver celebrates and honors the cultural

**[22.80s - 25.12s]**: and foundational contributions of indigenous people

**[25.12s - 29.84s]**: to our history, our past, our present and future.

**[29.84s - 0.00s]**: 

**[2.10s - 5.78s]**: and continues to promote the education of the Denver community about these historic

**[5.78s - 8.48s]**: and contemporary contributions of indigenous people.

**[10.26s - 13.84s]**: Section two, that the City and County of Denver,

**[13.84s - 18.68s]**: Colorado does hereby observe October 9th, 2017

**[18.68s - 20.84s]**: as Indigenous Peoples Day.

**[20.84s - 24.16s]**: Section three, that the clerk of the City and County

**[24.16s - 26.04s]**: of Denver shall attest and affix the seal

**[26.04s - 29.12s]**: of the City and County of Denver to this proclamation

**[29.12s - 0.00s]**: 

**[4.86s - 7.10s]**: and that a copy be transmitted, excuse me, to the Denver American Indian Commission,

**[7.10s - 9.82s]**: the City and County of Denver School District number one,

**[9.82s - 12.12s]**: and the Colorado Commission on Indian Affairs.

**[13.70s - 14.90s]**: Thank you, Councilman Lopez.

**[14.90s - 16.62s]**: Your motion to adopt.

**[16.62s - 19.94s]**: Mr. President, I move that proclamation number 1127,

**[19.94s - 22.78s]**: series of 2017 be adopted.

**[22.78s - 24.50s]**: All right, it has been moved and seconded.

**[24.50s - 25.62s]**: It comes by members of council.

**[25.62s - 26.86s]**: Councilman Lopez.

**[26.86s - 28.22s]**: Thank you, Mr. President.

**[29.14s - 0.00s]**: 

**[6.16s - 11.56s]**: It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples' Day in Denver, officially for the second time.

**[13.16s - 21.20s]**: It's always awesome to be able to see not just this proclamation come by my desk,

**[21.20s - 27.12s]**: but to see there's so many different people from our community in our council chambers.

**[28.96s - 0.00s]**: 

**[7.00s - 13.20s]**: It was a very beautiful piece of artwork that you presented to us earlier and it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created

**[13.20s - 18.48s]**: Indigenous Peoples Day when we sat down and wrote it and as a community we couldn't think

**[18.48s - 24.20s]**: of anything else to begin except for the confluence of the two rivers and those confluence of

**[24.20s - 27.48s]**: the two rivers created such a great city.

**[27.48s - 0.00s]**: 

**[6.44s - 9.90s]**: And we live in such an amazing city, and we're all proud of it, and sometimes we, and a lot of people from all over the country or all over the world are proud of it, and sometimes

**[9.90s - 18.92s]**: a little too proud of it is telling them to go back home, but I'm kidding when I say that.

**[18.92s - 24.28s]**: But the really nice thing about this is that we are celebrating Indigenous Peoples' Day

**[24.28s - 0.00s]**: 

**[5.80s - 11.48s]**: out of pride for who we are, who we are as a city and the contributions of indigenous people to the city, not out of spite, not out of a replacement of one culture over the

**[11.48s - 18.52s]**: other or out of contempt or disrespect.

**[18.52s - 26.64s]**: I think of a quote that Cesar Chavez made very popular and it stuck with me for a very

**[26.64s - 0.00s]**: 

**[7.32s - 14.32s]**: long time, and anytime I have the opportunity to speak in front of children, and especially children in our community that, you know, they often second-guess themselves on where

**[14.32s - 20.28s]**: they're coming from, who they are, and I always say that, you know, it's very important to

**[20.28s - 22.32s]**: be proud of who you're from.

**[22.32s - 27.60s]**: And the quote that I use from Cesar Chavez is, you know, pride in one's own culture does

**[27.60s - 0.00s]**: 

**[4.88s - 6.56s]**: does not require contempt or disrespect of another, right? And that's very important.

**[6.56s - 8.60s]**: It's very important for us to recognize that,

**[8.60s - 11.96s]**: no matter who we are, where we come from in this society,

**[11.96s - 13.80s]**: that your pride in your own culture

**[13.80s - 18.00s]**: does not require the contempt or disrespect of another.

**[18.00s - 22.44s]**: And man, what a year for that to just sit on our shoulders

**[22.44s - 25.76s]**: for a while, for us to think about, right?

**[25.76s - 0.00s]**: 

**[5.00s - 6.88s]**: And so I wanted to thank you all, I think the commission.

**[8.28s - 9.24s]**: There's gonna be a couple individuals that are gonna come speak.

**[9.24s - 11.36s]**: Thank you for your art, your lovely artwork,

**[11.36s - 13.80s]**: for us to see what's in your heart

**[13.80s - 16.48s]**: and what now has become, probably is gonna be

**[16.48s - 20.12s]**: a very important symbol for the community.

**[20.12s - 24.04s]**: And also just for the work, the daily work every single day.

**[24.04s - 25.80s]**: We still have a lot of brothers and sisters

**[25.80s - 29.22s]**: whose ancestors once lived in these lands freely

**[29.22s - 0.00s]**: 

**[6.16s - 11.32s]**: now stand on street corners, in poverty, without access to services, without access to sobriety, or even housing,

**[11.32s - 12.60s]**: or jobs.

**[12.60s - 18.48s]**: And what a cruel way to pay back a culture that

**[18.48s - 22.64s]**: has paved the way for the city to be built upon its shores.

**[22.64s - 26.04s]**: So we have a lot of work to do.

**[26.04s - 29.28s]**: And these kind of proclamations in this day is not a day off,

**[29.28s - 0.00s]**: 

**[2.60s - 5.16s]**: but it's a day on in Denver, right? And addressing those critical issues.

**[5.16s - 10.16s]**: So, I know that my colleagues are very supportive.

**[10.36s - 12.44s]**: I'm gonna ask you to support this proclamation,

**[12.44s - 14.60s]**: as I know you always have done in the past.

**[14.60s - 16.62s]**: I'm very proud of today.

**[16.62s - 20.16s]**: Oh, and we made Time Magazine and Newsweek once again,

**[20.16s - 22.48s]**: today, as being a leader in terms of the cities

**[22.48s - 25.36s]**: that are celebrating Indigenous Peoples Day.

**[25.36s - 27.20s]**: I wanted to make a point out of that.

**[28.24s - 0.00s]**: 

**[4.20s - 5.20s]**: Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Martega.

**[5.20s - 7.80s]**: Mr. President, I want to ask that my name be added.

**[7.80s - 14.80s]**: I don't think I could add much more to what Councilman Lopez has shared with us.

**[14.80s - 20.92s]**: I want to thank him for bringing this forward and really just appreciate all the contributions

**[20.92s - 29.40s]**: that our Native American community has contributed to this great city and great state.

**[29.40s - 0.00s]**: 

**[2.40s - 5.26s]**: I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created

**[5.26s - 8.28s]**: and had the benefit of being able to go down

**[8.28s - 13.28s]**: to the Four Corners for a peace treaty signing ceremony

**[13.70s - 15.70s]**: between the Utes and the Comanches

**[15.70s - 18.08s]**: that had been sort of at odds with each other

**[18.08s - 21.44s]**: for about 100 years and just being able to participate

**[21.44s - 23.80s]**: in that powwow was pretty awesome.

**[23.80s - 28.22s]**: So, and for those of you who continue to participate

**[28.22s - 0.00s]**: 

**[3.98s - 8.98s]**: in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions

**[9.28s - 10.12s]**: of the culture.

**[10.12s - 13.96s]**: I mean, to see that the dance continues to be carried on

**[13.96s - 17.88s]**: as well as the native language from generation to generation

**[17.88s - 22.40s]**: is just so incredible because in so many cultures,

**[22.40s - 27.40s]**: people have come here and assimilated to the norms here

**[29.82s - 0.00s]**: 

**[7.80s - 8.08s]**: You know the norms here, and they lose their language and and and lose a lot of the culture and in the native community

**[12.58s - 15.36s]**: That that hasn't happened that has that you know

**[21.20s - 21.72s]**: Commitment to just passing that on from generation to generation is is so important and

**[25.28s - 27.00s]**: So I'm happy to be a co-sponsor of this tonight. Thank you

**[0.00s - 0.84s]**: Thank you.

**[2.24s - 3.88s]**: All right, thank you, Councilwoman Ortega.

**[3.88s - 5.60s]**: Councilwoman Kenich.

**[5.60s - 8.12s]**: Thank you very much, and I also wanna thank my colleague

**[8.12s - 9.56s]**: for bringing this forward.

**[9.56s - 12.36s]**: And I just wanted to say a word to the artists

**[12.36s - 15.68s]**: about how beautiful and moving I thought this logo was

**[15.68s - 17.60s]**: and your description of it.

**[17.60s - 20.88s]**: And I think one of the things that is clear

**[20.88s - 24.40s]**: is the words sometimes don't convey the power of imagery

**[24.40s - 27.00s]**: or music or the other pieces that make up culture,

**[27.00s - 28.96s]**: and so I think the art is so important.

**[0.00s - 1.60s]**: And when you talked about water,

**[1.60s - 3.14s]**: I was also thinking about land,

**[3.14s - 5.74s]**: and I guess I just wanted to say thank you.

**[5.74s - 8.20s]**: Many of the Native American peoples of Colorado

**[8.20s - 10.08s]**: have been at the forefront, or actually nationally,

**[10.08s - 13.60s]**: of defending some of the public lands

**[13.60s - 15.48s]**: that have been protected over the last few years

**[15.48s - 17.44s]**: that are under attack right now.

**[17.44s - 20.42s]**: And they're places that the communities

**[20.42s - 23.48s]**: have fought to protect, but that everyone gets to enjoy.

**[23.48s - 26.32s]**: And so I just think that it's an example

**[26.32s - 29.16s]**: of where cultural preservation intersects

**[29.16s - 0.00s]**: 

**[3.00s - 6.84s]**: with environmental protection, with recreation, and all of the other ways that public lands are so important.

**[6.84s - 9.60s]**: And so I think I just wanted to say thank you for that,

**[9.60s - 12.24s]**: because I think we have some very sacred places

**[12.24s - 14.12s]**: in our country that are at risk right now.

**[14.12s - 16.88s]**: And so as we celebrate,

**[16.88s - 18.24s]**: I appreciate that there's still

**[18.24s - 19.92s]**: a piece of resistance in here.

**[19.92s - 23.00s]**: And I think that I just wanna mention a solidarity,

**[23.00s - 26.16s]**: and mention a feeling of solidarity with that resistance.

**[26.16s - 29.60s]**: So thank you, and happy Confluence Week.

**[0.72s - 7.52s]**: Thank you councilman can each and see no other comments. I'll just say a couple and

**[8.88s - 11.36s]**: in a time of such divisive

**[12.96s - 16.68s]**: Ugliness and just despicable behavior from our leadership

**[18.20s - 22.92s]**: The reason I'm so supportive of indigenous people's days because it means inclusivity

**[23.08s - 27.44s]**: It means respecting all respecting those who have been silenced

**[29.36s - 0.00s]**: 

**[8.60s - 14.04s]**: on purpose for a long time and whose history has not been told, and so we celebrate inclusivity in the face of such evil times, honestly.



In [34]:
# Set pandas option to display full column content
pd.set_option("display.max_colwidth", None)

# Now re-create or display your DataFrame
df = pd.DataFrame(result["chunks"])
df.columns = ["Timestamps", "Text"]
df


,Timestamps,Text
0,"(0.0, 4.72)","and kind of the confluence of this whole idea of the Confluence Week, the merging of two"
1,"(4.72, 5.72)",rivers.
2,"(5.72, 13.72)","And as we've kind of seen recently in politics and in the world, there's a lot of situations"
3,"(13.72, 16.96)","where water is very important right now, and it's a very big issue."
4,"(16.96, 21.96)",So that is the reason that the back of the logo is considered water.
...,...,...
245,"(12.96, 16.68)",Ugliness and just despicable behavior from our leadership
246,"(18.2, 22.92)",The reason I'm so supportive of indigenous people's days because it means inclusivity
247,"(23.08, 27.44)",It means respecting all respecting those who have been silenced
248,"(29.36, 0.0)",


In [35]:
from collections import Counter
import re

words = re.findall(r'\b\w+\b', result["text"].lower())
common_words = Counter(words).most_common(10)
print("🔡 Top 10 most frequent words:")
for word, count in common_words:
    print(f" - {word}: {count}")

🔡 Top 10 most frequent words:
 - the: 120
 - and: 115
 - of: 106
 - to: 62
 - that: 54
 - a: 41
 - i: 40
 - you: 35
 - in: 32
 - so: 26


In [36]:
import datetime

word_count = len(result["text"].split())
duration = result["chunks"][-1]["timestamp"][1]  # Last timestamp end
print(f"🗣️  Word count: {word_count}")
print(f"⏱️  Approx. audio duration: {str(datetime.timedelta(seconds=round(duration)))}")

🗣️  Word count: 2162
⏱️  Approx. audio duration: 0:00:14
